# Text classification

The task concentrates on content-based text classification.


## Tasks

1. Get acquainted with the data of the [Polish Cyberbullying detection dataset](https://huggingface.co/datasets/poleval2019_cyberbullying). 
   Pay special attention to the distribution of the positive and negative examples in the first task as well as
   distribution of the classes in the second task.


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import *
from sklearn import metrics
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

# print(list_datasets())

dataset = load_dataset('poleval2019_cyberbullying','task01')
print(dataset['test'][3])
print(dataset['train'][3])

# # List all the available metrics
# print(list_metrics())

# # Load a metric
# squad_metric = load_metric('squad')

# # Process the dataset - add a column with the length of the context texts
# dataset_with_length = squad_dataset.map(lambda x: {"length": len(x["context"])})

# # Process the dataset - tokenize the context texts (using a tokenizer from the 🤗 Transformers library)
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# tokenized_dataset = squad_dataset.map(lambda x: tokenizer(x['context']), batched=True)


Reusing dataset poleval2019_cyber_bullying (/home/x/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': '@anonymized_account @anonymized_account Przecież to nawet nie jest przewrotka 😂', 'label': 0}
{'text': '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.', 'label': 0}


2. Train the following classifiers on the training sets (for the task 1 and the task 2)


    i. Bayesian classifier with TF * IDF weighting.
    ii. Fasttext text classifier
    iii. Transformer classifier (take into account that a number of experiments should be performed for this model).

In [89]:
train_X = dataset['train']['text']
train_y = dataset['train']['label']

test_X = dataset['test']['text']
test_y = dataset['test']['label']

tf_idf = TfidfVectorizer()


X_train_tf = tf_idf.fit_transform(train_X)
X_train_tf = tf_idf.transform(train_X)
X_test_tf = tf_idf.transform(test_X)

print("n_samples: %d, n_features: %d" % X_train_tf.shape)
print(X_test_tf.shape)

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf,train_y)

y_pred = naive_bayes_classifier.predict(X_test_tf)
y_pred
X_train_tf

print(metrics.classification_report(test_y,y_pred,target_names=['Positive','Negative']))

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(dataset['train'])
# vectorizer.get_feature_names_out()

print("Confusion matrix")
print(metrics.confusion_matrix(test_y,y_pred))

n_samples: 10041, n_features: 22872
(1000, 22872)
              precision    recall  f1-score   support

    Positive       0.87      1.00      0.93       866
    Negative       1.00      0.01      0.01       134

    accuracy                           0.87      1000
   macro avg       0.93      0.50      0.47      1000
weighted avg       0.88      0.87      0.81      1000

Confusion matrix
[[866   0]
 [133   1]]


In [92]:
# Sformułowań użyto jedynie do celów naukowych i nie odzwierciedlają one poglądów studenta
testing_X = [x for (x,y)in zip(test_X,test_y) if y==1][1:10]
testing_y = [y for (x,y)in zip(test_X,test_y) if y==1][1:10]
testing_X.append("@anonymized_account A na drzewach zamiast liści będą wisieć syjoniści")
testing_y.append(1)
test_input = tf_idf.transform(testing_X)
result = naive_bayes_classifier.predict(test_input)
# print(list(zip(testing_X,testing_y,result)))
for X,y, result in zip(testing_X,testing_y,result):
    print(f'true={y}, predicted={result}, {X}')


true=1, predicted=0, @anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości
true=1, predicted=0, @anonymized_account Adrian Juda, figurant WSI i  lobby żydowskiego
true=1, predicted=0, @anonymized_account Widać ludziom w Sączu tak pasuje. Ja bym mu w gębę napluł
true=1, predicted=0, @anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli
true=1, predicted=0, RT @anonymized_account @anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli
true=1, predicted=0, @anonymized_account @anonymized_account @anonymized_account Najbardziej to on jest wolny od mózgu.
true=1, predicted=0, @anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytaryzm!
true=1, predicted=0, RT @anonymized_account @anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytaryzm!
true=1, predicted=0, @anonymized_account @anonymized_account @anonymized_account Podstawowe zadanie każdego ksi

In [85]:
testing_X

['@anonymized_account Dokładnie, pisdzielstwo nie ma prawa rozpierdalać systemu,  sądownictwa nie mając większości',
 '@anonymized_account Adrian Juda, figurant WSI i  lobby żydowskiego',
 '@anonymized_account Widać ludziom w Sączu tak pasuje. Ja bym mu w gębę napluł',
 '@anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli',
 'RT @anonymized_account @anonymized_account powinnaś odpowiedzieć za działanie na szkodę Polski i obywateli',
 '@anonymized_account @anonymized_account @anonymized_account Najbardziej to on jest wolny od mózgu.',
 '@anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytaryzm!',
 'RT @anonymized_account @anonymized_account   Półgłówek Wieliński, wymyślił sobie półautorytaryzm!',
 '@anonymized_account @anonymized_account @anonymized_account Podstawowe zadanie każdego księdza to narzucanie wiary.']

   
   
3. Compare the results of classification on the test set. Select the appropriate measures (from accuracy, F1, macro/micro F1, MCC) to compare the results.

4. Select 1 TP, 1 TN, 1 FP and 1 FN from your predictions (for the best classifier) and compare the decisions of each
   classifier on these examples using [LIME](https://github.com/marcotcr/lime).

5. Answer the following questions:


1. Which of the classifiers works the best for the task 1 and the task 2.
1. Did you achieve results comparable with the results of [PolEval Task](http://2019.poleval.pl/index.php/results/)?
1. Did you achieve results comparable with the [Klej leaderboard](https://klejbenchmark.com/leaderboard/)?
1. Describe strengths and weaknesses of each of the compared algorithms.
1. Do you think comparison of raw performance values on a single task is enough to assess the value of a given
  algorithm/model?
1. Did SHAP show that the models use valuable features/words when performing their decision?